In [28]:
#############################################################
### LOAD MAILCHIMP DATA INTO MAILCHIMP STAGING TABLE CODE ###
### Version - Final                                       ###
### WRITTEN BY DANA CAINES - PROD DATE = 12/12/23         ###
#############################################################

import pandas as pd
import json
import mysql.connector
from mysql.connector import Error

# Function to check and return alumniID: if alumni does not exist, insert and keep alumniID for adding to row in stage table; else, if alumni exists, get alumniID and add to row in stage table
def get_or_create_alumni_id(firstname, lastname, cursor):
    # Check if alumni exists
    cursor.execute("SELECT alumniID FROM alumni WHERE firstname = %s AND lastname = %s", (firstname, lastname))
    result = cursor.fetchone()
    if result:
        return result[0]
    else:
        return None
        # Insert new alumni record
        #cursor.execute("INSERT INTO alumni (firstname, lastname) VALUES (%s, %s)", (firstname, lastname))
    return cursor.lastrowid


def convert_datetime_format(date_str):
    if date_str and date_str != 'NULL':
        return pd.to_datetime(date_str).strftime('%Y-%m-%d %H:%M:%S')
    return 'NULL'


#Function to insert record into stage table: if the record exists, then insert; else update the record
def update_insert_row(cursor, conn, data_tuple):
    # Check if the record already exists
    
    firstname = data_tuple[2]
    lastname = data_tuple[3]
    
    cursor.execute("""
        SELECT EXISTS(
            SELECT 1 FROM stg_in_mailchimp 
            WHERE  firstname = %s AND lastname = %s
        )
    """, ('firstname', 'lastname'))
    record_exists = cursor.fetchone()[0]

    if record_exists:
        pass
        
        #cursor.execute(update_stmt, update_data_tuple)
    else:
        # Insert new record
        insert_stmt = """
            INSERT INTO stg_in_mailchimp (
                alumniID,
                firstname, 
                lastname, 
                fullname, 
                email_address,  
                address1, 
                address2,
                city, 
                state,
                zip_code,
                country,
                phone, 
                mc_status,
                join_date
            )
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
       
    
    insert_data_tuple = data_tuple
    # print("Update Data Tuple Length:", len(insert_data_tuple))
        
    
    cursor.execute(insert_stmt, insert_data_tuple)
        

    conn.commit()
    #cursor.connection.commit()

# Function to read csv and insert into database stage table
def insert_csv_to_mc_stage(file_path, host_name, user_name, user_password, db_name):
    
    try:
        # Read the csv file
        df = pd.read_csv(file_path)
        #print(df.columns)
        
        df = df.where(pd.notnull(df), 'NULL')

        # Connect to the database
        conn = mysql.connector.connect(host=host_name, user=user_name, passwd=user_password, database=db_name)
        print('Connected to db')
        cursor = conn.cursor()

        # Iterate over the DataFrame and insert/update records
        for index, row in df.iterrows():
            try:
                row['join_date'] = convert_datetime_format(row['join_date'])
                
                alumni_id = get_or_create_alumni_id(row['firstname'], row['lastname'], cursor)
                if alumni_id:
                    data_tuple = (alumni_id,) + tuple(row[col] for col in df.columns)
                    update_insert_row(cursor, conn, data_tuple)
                    conn.commit()
                else:
                    pass
            except Exception as e:
                print(f"Error at dataframe index {index}: {e}")
                error_data_tuple = (str(e),) + (alumni_id,) + tuple(row[col] for col in df.columns)
                insert_error_log(cursor, conn, error_data_tuple, 'insert_csv_to_mc_stage')
                conn.rollback()
        
        print("Records were successfully processed")

    except Error as e:
        print("Error while connecting to MySQL", e)
    finally:
        if conn.is_connected():
            cursor.close()
            conn.close()
            print("MySQL connection is closed")

            
def insert_error_log(cursor, conn, error_data_tuple, source):
    try:
        insert_stmt = """
            INSERT INTO error_log (
            error_message, 
            source,
            alumni_id, 
            firstname, 
            lastname, 
            fullname,
            email_first,
            phone,
            address_1, 
            address_2,
            city, 
            state,
            zip_code,
            country,
            mc_status,
            mc_join_date 
        ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        
        error_tuple = (str(error_data_tuple[0]), source) + error_data_tuple[1:]
        cursor.execute(insert_stmt, error_tuple, )
        conn.commit()
        
    except Exception as e:
        print(f"Error logging failed: {e}")
        

# Identify credentials and working directory
csv_file_path = '/Users/dcain/Library/CloudStorage/OneDrive-Personal/Saint Marys Course/OPS 659 Practicum/Alumni DB Python Files/Staged_Files/clean_mailc_12Dec23.csv'
# MySQL connection details
host='localhost'
user='root'
passwd='rusercaines'
database='smc_alumni_test_2'
# host = '34.27.128.119'
# user = 'ahmed',
# passwd = 'abcdefgh',
# database = 'smc_alumni'

# Call the function to read CSV and insert into MySQL
insert_csv_to_mc_stage(csv_file_path, host, user, passwd, database)

Connected to db
Records were successfully processed
MySQL connection is closed
